# File to see graphics of default simulation

## Imports to collect data and plot graphics

In [ ]:
from components.graphic_generator import Graphic_Gen
from components.data_manager import Data_Manager
import components.network_data as nwd
from components.utils.enums import Colors
import numpy as np
import pandas as pd

## Constants

In [ ]:
PATH_TO_CSV: str = "data/default_simulation/default_simulation.csv"
NUMBER_OF_GRAPHIC_POINTS: int = 7
BLACK_HOLES_NUMBER: list = [1, 3, 5]
INTENSITIES: list = [intensity*0.1 for intensity in range(1, 8)]
COLORS_LIST: list[Colors] = [Colors.YELLOW, Colors.ORANGE, Colors.RED]

## Load csv

In [ ]:
dm: Data_Manager = Data_Manager()
loaded: bool = dm.load_csv(PATH_TO_CSV)
if not loaded:
    print("File don't loaded")

## Convert csv to pandas DataFrame

In [ ]:
dframe: pd.DataFrame = dm.csv_to_dataframe()

## Manipulate data to collect columns from default network (without black holes)

In [ ]:
default_consumed_eprs: pd.Series = (dframe.loc[dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0, nwd.CONSUMED_EPRS])
default_success_tax: pd.Series = (dframe.loc[dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0, nwd.TOTAL_REQUEST_SUCCESS])
default_avg_fidelity: pd.Series = (dframe.loc[dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0, nwd.TOTAL_ROUTE_FIDELITY])

default_number_requests: pd.Series = (dframe.loc[dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0, nwd.REQUESTS])

default_axle_consumed_eprs: np.ndarray = (default_consumed_eprs / default_number_requests).to_numpy()
default_axle_success_tax: np.ndarray = (default_success_tax / default_number_requests).to_numpy()
default_axle_avg_fidelity: np.ndarray = (default_avg_fidelity / default_number_requests).to_numpy()

## Function to collect attacked data from each number of black holes

In [ ]:
def collect_axis_list(column: str, targets: int) -> list[list[np.ndarray]]:
    attacked_axis_list: list[list[np.ndarray]] = []
    for bh_number in BLACK_HOLES_NUMBER:
        attacked_axis: list[np.ndarray] = []
        for i in range(1, NUMBER_OF_GRAPHIC_POINTS+1):  
            mask: pd.Series = (
                (dframe[nwd.INTENSITY] == f"i: {(i*0.1):.1f}") &
                (dframe[nwd.TARGETS_PER_BLACK_HOLE] == targets) &
                (dframe[nwd.NUMBER_OF_BLACK_HOLES] == bh_number)
            )
            attack_consumed_eprs: pd.Series = (dframe.loc[mask, column])
            attack_number_requests: pd.Series = (dframe.loc[mask, nwd.REQUESTS])
            data: np.ndarray = (attack_consumed_eprs / attack_number_requests).to_numpy()

            attacked_axis.append(data)
        attacked_axis_list.append(attacked_axis)
    
    return attacked_axis_list

## Use the function to collect axis list

In [ ]:
attacked_axis_list_consumed_eprs_no_targets: list[list[np.ndarray]] = collect_axis_list(column=nwd.CONSUMED_EPRS, targets=0)
attacked_axis_list_consumed_eprs_with_targets: list[list[np.ndarray]] = collect_axis_list(column=nwd.CONSUMED_EPRS, targets=1)

attacked_axis_list_success_tax_no_targets: list[list[np.ndarray]] = collect_axis_list(column=nwd.TOTAL_REQUEST_SUCCESS, targets=0)
attacked_axis_list_success_tax_with_targets: list[list[np.ndarray]] = collect_axis_list(column=nwd.TOTAL_REQUEST_SUCCESS, targets=1)

attacked_axis_list_avg_fidelity_no_targets: list[list[np.ndarray]] = collect_axis_list(column=nwd.TOTAL_ROUTE_FIDELITY, targets=0)
attacked_axis_list_avg_fidelity_with_targets: list[list[np.ndarray]] = collect_axis_list(column=nwd.TOTAL_ROUTE_FIDELITY, targets=1)

## Functions to transform all arrays in same size

In [ ]:
def find_min_length(list_of_attacked_axis_lists: list[list[list[np.ndarray]]], list_of_default_axis: list[np.ndarray]) -> int:
    min_length: int = -1
    for axis_list in list_of_attacked_axis_lists:
        for axle in axis_list:
            tmp_min_length: int = min(len(array) for array in axle)
            
            if min_length == -1:
                min_length: int = tmp_min_length 
                continue

            if tmp_min_length < min_length:
                min_length = tmp_min_length
    
    for default_axle in list_of_default_axis:
        if min_length > len(default_axle):
            min_length = len(default_axle)
    
    return min_length

def resize_attacked_array_to_same_length(min_length: int, attacked_axis_list: list[list[np.ndarray]]) -> list[np.ndarray]:
    attacked_axis: list[np.ndarray] = []
    for axis in attacked_axis_list:
        axle: np.ndarray = np.array([array[:min_length] for array in axis])
        attacked_axis.append(axle)

    return attacked_axis

def resize_default_array_to_same_length(min_length: int, default_axle: np.ndarray):
    return default_axle[:min_length]

## Transform all arrays in same length

In [ ]:
all_attacked_arrays_list = [
    attacked_axis_list_avg_fidelity_no_targets, 
    attacked_axis_list_avg_fidelity_with_targets,
    attacked_axis_list_consumed_eprs_no_targets,
    attacked_axis_list_consumed_eprs_with_targets,
    attacked_axis_list_success_tax_no_targets,
    attacked_axis_list_success_tax_with_targets
]

all_default_arrays_list = [
    default_axle_avg_fidelity,
    default_axle_consumed_eprs,
    default_axle_success_tax
]

# find min length
min_length: int = find_min_length(all_attacked_arrays_list, all_default_arrays_list)

# resize attacked lists
attacked_axis_eprs_consumed_no_targets = resize_attacked_array_to_same_length(min_length, attacked_axis_list_consumed_eprs_no_targets)
attacked_axis_eprs_consumed_with_targets = resize_attacked_array_to_same_length(min_length, attacked_axis_list_consumed_eprs_with_targets)

attacked_axis_success_tax_no_targets = resize_attacked_array_to_same_length(min_length, attacked_axis_list_success_tax_no_targets)
attacked_axis_success_tax_with_targets = resize_attacked_array_to_same_length(min_length, attacked_axis_list_success_tax_with_targets)

attacked_axis_avg_fidelity_no_targets = resize_attacked_array_to_same_length(min_length, attacked_axis_list_avg_fidelity_no_targets)
attacked_axis_avg_fidelity_with_targets = resize_attacked_array_to_same_length(min_length, attacked_axis_list_avg_fidelity_with_targets)

# resize default axis
default_axle_avg_fidelity = resize_default_array_to_same_length(min_length, default_axle_avg_fidelity)
default_axle_consumed_eprs = resize_default_array_to_same_length(min_length, default_axle_consumed_eprs)
default_axle_success_tax = resize_default_array_to_same_length(min_length, default_axle_success_tax)

## Create a graphic generator

In [ ]:
graphic_generator: Graphic_Gen = Graphic_Gen()

## Add data to graphic gen: Consumed Eprs without targets

In [ ]:
graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = np.array(attacked_axis_eprs_consumed_no_targets[i])
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=False, default_axle=default_axle_consumed_eprs, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=False, default_axle=default_axle_consumed_eprs, color=COLORS_LIST[i])

## Plot graphic

In [ ]:
graphic_generator.plot(tittle="Diff Eprs Consummed (target off)", x_label="Intensity", y_label="Diff Eprs per Request", grid=True)

## Consumed Eprs with target

In [ ]:
graphic_generator = Graphic_Gen()

graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = np.array(attacked_axis_eprs_consumed_with_targets[i])
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=False, default_axle=default_axle_consumed_eprs, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=False, default_axle=default_axle_consumed_eprs, color=COLORS_LIST[i])

graphic_generator.plot(tittle="Diff Eprs Consummed (target on)", x_label="Intensity", y_label="Diff Eprs per Request", grid=True)

## Success tax without target

In [ ]:
graphic_generator = Graphic_Gen()

graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = np.array(attacked_axis_success_tax_no_targets[i])
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=False, default_axle=default_axle_success_tax, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=False, default_axle=default_axle_success_tax, color=COLORS_LIST[i])

graphic_generator.plot(tittle="Diff Success Tax (target off)", x_label="Intensity", y_label="Diff Success Tax", grid=True)

## Success tax with target

In [ ]:
graphic_generator = Graphic_Gen()

graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = np.array(attacked_axis_success_tax_with_targets[i])
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=False, default_axle=default_axle_success_tax, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=False, default_axle=default_axle_success_tax, color=COLORS_LIST[i])

graphic_generator.plot(tittle="Diff Success Tax (target on)", x_label="Intensity", y_label="Diff Success Tax", grid=True)

## Average fidelity route without target

In [ ]:
graphic_generator = Graphic_Gen()

graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = np.array(attacked_axis_avg_fidelity_no_targets[i])
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=False, default_axle=default_axle_avg_fidelity, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=False, default_axle=default_axle_avg_fidelity, color=COLORS_LIST[i])

graphic_generator.plot(tittle="Diff Avg Fidelity (target off)", x_label="Intensity", y_label="Diff Avg Fidelity", grid=True)

## Average fidelity route with target

In [ ]:
graphic_generator = Graphic_Gen()

graphic_generator.define_x_axle(np.array(INTENSITIES))

for i, bh_number in enumerate(BLACK_HOLES_NUMBER):
    y_axle: np.ndarray = np.array(attacked_axis_avg_fidelity_with_targets[i])
    if bh_number == 1:
        graphic_generator.append_y_axle(y_axle, label="1 node", standard_deviation=False, default_axle=default_axle_avg_fidelity, color=COLORS_LIST[i])
    else:
        graphic_generator.append_y_axle(y_axle, label=f"{bh_number} nodes", standard_deviation=False, default_axle=default_axle_avg_fidelity, color=COLORS_LIST[i])

graphic_generator.plot(tittle="Diff Avg Fidelity (target on)", x_label="Intensity", y_label="Diff Avg Fidelity", grid=True)